In [1]:
# Setting up the environment
import pandas as pd
import numpy as np
from urllib.request import urlopen
import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install lxml
from bs4 import BeautifulSoup

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
     |████████████████████████████████| 5.8MB 1.1MB/s eta 0:00:01     |██                              | 378kB 1.1MB/s eta 0:00:05


In [3]:
# Using bs4 to import html to dataframe
url = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["Postalcode", "Borough","Neighbourhood"])

In [4]:
# Removing unnecessary rows
df = df[df.Borough != 'Not assigned']
df = df[df.Borough != 'None']

In [5]:
# Converting values to str for subsequent concatenation
df['Postalcode'] = df['Postalcode'].astype(str)
df['Borough'] = df['Borough'].astype(str)
df['Neighbourhood'] = df['Neighbourhood'].astype(str)

In [6]:
# Concatenation and removing extra '\n' from data values
df_new = df.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_new = df_new.replace('\n',' ', regex=True)
df_new.head(10)

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff ,Cliffside West"


In [7]:
# Setting not assigned values in neighbourhood to follow borough value
df_new.Neighbourhood = df_new.Borough.where(df_new.Neighbourhood == 'Not assigned ', df_new.Neighbourhood)

In [ ]:
# Double-check final result, print number of rows
print(df_new.to_string())
df_new.shape

In [11]:
df_new.to_csv("output.csv", index=False)